# NLP Machine Learning Classifiers

## Building a Basic Random Forest Model

In [4]:
# Import the dependencies
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

### Read in & Clean Text Data

In [5]:
# Read in the text data
data = pd.read_csv("./data/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

In [6]:
# Defind stopwords and Porter Stemmer
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [8]:
# Create count_punct() function
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

In [9]:
# Create clean_text() function
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [10]:
# Create new columns for body length and punctuation %
data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

In [12]:
# Clean the dataframe by TF-IDF Vectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

In [13]:
# Concat the dataframes
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore Random Forest Classifier Attributes & Hyperparameters

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0

### Explore Random Forest Classifier through Cross Validation

In [16]:
from sklearn.model_selection import KFold, cross_val_score

In [17]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.96588869, 0.97486535, 0.97574124, 0.95867026, 0.96675651])

## Building Random Forest Model with Grid-Search

**Grid-Search**: Exhaustively search all parameter combinations in a given grid to determine the best model.

### Building the Grid-Search

In [18]:
# Import the sklearn library
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [19]:
# Split the data into 80/20 raito for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [20]:
# Create a function for training the random forest model
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [21]:
# Use a for loop to calculate the precision, recall, and accuracy rate 
# with different combinations of parameters
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.275 / Accuracy: 0.896
Est: 10 / Depth: 20 ---- Precision: 0.978 / Recall: 0.569 / Accuracy: 0.936
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.688 / Accuracy: 0.955
Est: 10 / Depth: None ---- Precision: 0.992 / Recall: 0.762 / Accuracy: 0.965
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.3 / Accuracy: 0.899
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.594 / Accuracy: 0.942
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.669 / Accuracy: 0.952
Est: 50 / Depth: None ---- Precision: 0.992 / Recall: 0.769 / Accuracy: 0.966
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.269 / Accuracy: 0.895
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.6 / Accuracy: 0.943
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.688 / Accuracy: 0.955
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.794 / Accuracy: 0.97


## Evaluate Random Forest Model with Grid-Search Cross Validation

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Exploring parameter settings using GrdSearchCV

In [26]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [24]:
# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

In [27]:
# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

In [28]:
# Create the Random Forest Classifier and a set of parameters
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

# Create the Grid Search Cross Validation and a summary table for TF-IDT
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,53.348344,1.546225,0.723465,0.124154,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.978437,0.974843,0.969452,0.972147,0.974493,0.003350,1
6,5.060269,0.384639,0.192485,0.046288,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.980269,0.974843,0.975741,0.968553,0.971249,0.974133,0.004006,2
7,28.423598,0.869595,0.501459,0.170448,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.975785,0.977538,0.974843,0.969452,0.972147,0.973954,0.002846,3
11,49.695312,0.634948,0.335827,0.040256,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977578,0.978437,0.972147,0.967655,0.971249,0.973415,0.004050,4
10,28.617675,0.391918,0.359838,0.070942,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.974843,0.973944,0.966757,0.969452,0.972157,0.003466,5


In [29]:
# Create the Random Forest Classifier and a set of parameters
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

# Create the Grid Search Cross Validation and a summary table for Count Vectorizer
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,47.411020,0.517368,0.363028,0.024850,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976682,0.975741,0.973944,0.970350,0.969452,0.973235,0.002874,1
7,27.155185,0.901296,0.424465,0.093851,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978475,0.973046,0.973944,0.966757,0.971249,0.972696,0.003808,2
8,51.762185,1.180447,0.869474,0.517071,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.972147,0.973046,0.967655,0.972147,0.972337,0.002877,3
10,27.680381,0.632318,0.339891,0.089652,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978475,0.972147,0.973046,0.965858,0.971249,0.972157,0.004031,4
3,5.029351,0.361534,0.215224,0.016989,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.971300,0.972147,0.970350,0.965858,0.973944,0.970720,0.002703,5


## Explore Gradient Boosting Model with Grid-Search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Explore Gradient Boosting Classifier Attributes and Hyperparameters

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_initialized', '_check_params', '_clear_state', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_raw_predict', '_raw_predict_init', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params', 'staged_decision_function', 'staged_predict', 'staged_p

### Building the Grid-Search

In [32]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [33]:
# Split the data into 80/20 raito for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [34]:
# Create function for training with Gradient Boosting
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [35]:
# Create for loop to run the model with different parameters
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\lokma\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.866
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.963 / Recall: 0.698 / Accuracy: 0.956
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.897 / Recall: 0.758 / Accuracy: 0.956
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.013 / Accuracy: 0.868
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.922 / Recall: 0.799 / Accuracy: 0.964
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.898 / Recall: 0.772 / Accuracy: 0.958
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.013 / Accuracy: 0.868
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.94 / Recall: 0.846 / Accuracy: 0.972
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.915 / Recall: 0.792 / Accuracy: 0.962
Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.867
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.9 / Recall: 0.846 / Accuracy: 0.967
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.916 / Recall: 0.805 / Accurac

KeyboardInterrupt: 

## Evaluate Gradient Boosting with Grid-Search Cross Validation

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Exploring parameter settings using GridSearchCV

In [ ]:
# Import the dependencies
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create the Gradient Boosting Classifier and a set of parameters
gb = GradientBoostingClassifier()
param = {'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15], 'learning_rate': [0.1]}

# Create the Grid Search Cross Validation and a summary table for TF-IDF
clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
# Create the Gradient Boosting Classifier and a set of parameters
gb = GradientBoostingClassifier()
param = {'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15], 'learning_rate': [0.1]}

# Create the Grid Search Cross Validation and a summary table for Count Vectorizer
clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

## Model Selection

### Split the Data into Training and Testing Set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Vectorize Text

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

X_train_vect.head()

### Final Evaluation of Models

In [ ]:
# Import the dependencies
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [ ]:
# Evaluate the Random Forest Model
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

rf_model = rf.fit(X_train_vect, y_train)
y_pred = rf_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

In [ ]:
# Evaluate the Gradient Boosting Model
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

gb_model = gb.fit(X_train_vect, y_train)
y_pred = gb_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))